In [1]:
!pip install mesa==2.3.1 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 82.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.9/107.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 425.7/425.7 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.2/168.2 kB 11

In [1]:
from mesa import Agent, Model
from mesa.space import SingleGrid
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector
from mesa.batchrunner import batch_run

import numpy as np
import pandas as pd

In [2]:
class SimAgent(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.actionPoints = 4
        self.extraActionPoints = 0
        self.isCarryingVictim = False
        self.victimsRescued = 0
        self.actionPointsUsed = 1
        self.isKnockedOut = False

    def move(self, target):
        """Moverse a una celda adyacente válida."""
        if self.actionPoints > 0 and self.model.grid.is_cell_empty(target):
            fire_cost = 2 if self.model.cells[target] == 2 else 1
            if self.actionPoints >= fire_cost:
                self.model.grid.move_agent(self, target)
                self.actionPoints -= fire_cost

    def extinguish(self):
        """Extinguir fuego o humo en la celda actual."""
        cell_status = self.model.cells[self.pos]
        if cell_status == 2:  # Fuego
            self.model.cells[self.pos] = 1  # Convertir a humo
            self.actionPoints -= 1
        elif cell_status == 1:  # Humo
            self.model.cells[self.pos] = 0  # Eliminar humo
            self.actionPoints -= 1

    def rescue(self):
        """Rescatar víctima en la celda actual."""
        if self.model.cells[self.pos] == 3:  # POI revelado como víctima
            self.isCarryingVictim = True
            self.model.cells[self.pos] = 0
            self.actionPoints -= 2

    def chop(self, direction):
        """Destruir una pared en la dirección indicada."""
        if self.actionPoints >= 2:
            wall = self.model.walls[self.pos]
            if wall[direction] < 2:
                wall[direction] += 1
                self.actionPoints -= 2

    def step(self):
        """Acciones del agente en cada paso."""
        if self.isCarryingVictim:
            self.move_towards(self.model.exitPos)
            if self.pos == self.model.exitPos:
                self.victimsRescued += 1
                self.isCarryingVictim = False
        else:
            self.search_and_rescue()

    def move_towards(self, target):
        """Moverse hacia una celda objetivo."""
        dx = target[0] - self.pos[0]
        dy = target[1] - self.pos[1]
        move = (self.pos[0] + np.sign(dx), self.pos[1] + np.sign(dy))
        self.move(move)

    def search_and_rescue(self):
        """Buscar víctimas o extinguir fuego cercano."""
        neighbors = self.model.grid.get_neighborhood(self.pos, moore=False)
        for pos in neighbors:
            if self.model.cells[pos] == 3:  # POI
                self.move(pos)
                self.rescue()
                break
            elif self.model.cells[pos] == 2:  # Fuego
                self.extinguish()
                break


In [3]:
def getGrid(model):
    """
    Devuelve el estado actual del tablero como una matriz numérica.
    - 0: Celda vacía
    - 1: Humo
    - 2: Fuego
    - 3: Víctima
    - 4: Falsa alarma
    - 5: Agente
    """
    gridState = np.zeros((model.grid.width, model.grid.height))
    for (x, y) in model.grid.coord_iter():
        # Estado base de la celda
        gridState[x][y] = model.cells[x][y]
        # Si hay un agente, sobrescribir el estado
        if model.grid[x][y] is not None:
            gridState[x][y] = 5
    return gridState


In [ ]:
class FireRescueModel(Model):
    def __init__(self, walls, markers, fires, doors, entry_points):
        super().__init__()
        self.grid = SingleGrid(6, 8, torus=False)
        self.schedule = RandomActivation(self)
        self.cells = np.zeros((6, 8))
        self.walls = walls
        self.entry_points = entry_points
        self.exitPos = (5, 7)  # Salida predefinida
        self.damage = 0
        self.victims_rescued = 0
        self.victims_lost = 0
        self.steps = 0
        self.datacollector = DataCollector(
            model_reporters={
                "Grid": getGrid,
                "Steps": lambda model: model.steps,
                "Damage": lambda model: model.damage,
                "VictimsRescued": lambda model: model.victims_rescued,
                "VictimsLost": lambda model: model.victims_lost
            }
            ,agent_reporters = {"VictimsRescued": lambda agent: agent.victimsRescued,
                "ActionPointsUsed": lambda agent: agent.actionPointsUsed,
                "Efficiency": lambda agent: agent.victimsRescued / agent.actionPointsUsed}
        )
        
        self.startFire(fires)
        self.placeAgents(entry_points)

    def startFire(self, fires):
        placed = 0
        while placed < len(fires):
            x,y = np.random.randint(6), np.random.randint(8)
            if self.cells[x][y] == 0:
                self.cells[x][y] = 2
                placed += 1

    def placeAgents(self, entry_points):
        for i in range(len(entry_points)):
            agent = SimAgent(i, self)
            self.grid.move_to_empty(agent)
            self.schedule.add(agent)


    def advance_fire(self):
        """Propagación del fuego."""
        target = (np.random.randint(6), np.random.randint(8))
        if self.cells[target] == 1:  # Humo
            self.cells[target] = 2
        elif self.cells[target] == 2:  # Fuego
            self.handle_explosion(target)

    def handle_explosion(self, pos):
        """Manejar explosiones y daños estructurales."""
        neighbors = self.grid.get_neighborhood(pos, moore=False)
        for neighbor in neighbors:
            if self.cells[neighbor] == 0:
                self.cells[neighbor] = 2
            elif self.cells[neighbor] == 1:
                self.cells[neighbor] = 2
            elif self.cells[neighbor] == 3:  # Víctima
                self.cells[neighbor] = 0
                self.victims_lost += 1

    def step(self): 
        if not self.check_game_end():
            self.steps += 1
        self.datacollector.collect(self)
        self.schedule.step()

    def check_game_end(self):
        """Verificar condiciones de fin del juego."""
        if self.victims_rescued >= 7:
            return True
        elif self.victims_lost >= 4 or self.damage >= 24:
            return True
        return False

In [8]:
def load_scenario(file_path):
    """
    Procesa un archivo de texto con la configuración inicial del escenario y retorna los datos necesarios.
    """
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Leer la cuadrícula del escenario con las paredes
    walls = []
    for i in range(6):  # 6 filas de celdas
        row = lines[i].strip().split()
        walls.append([list(map(int, cell)) for cell in row])

    walls = np.array(walls)  # Convertir a numpy array para facilitar operaciones

    # Leer los marcadores de puntos de interés
    markers = []
    for i in range(6, 9):  # 3 líneas de marcadores
        line = lines[i].strip().split()
        row, col, marker_type = int(line[0]), int(line[1]), line[2]
        markers.append((row - 1, col - 1, marker_type))  # Ajustar índices a 0-based

    # Leer los marcadores de fuego
    fires = []
    for i in range(9, 19):  # 10 líneas de marcadores de fuego
        line = lines[i].strip().split()
        row, col = int(line[0]), int(line[1])
        fires.append((row - 1, col - 1))  # Ajustar índices a 0-based

    # Leer las puertas
    doors = []
    for i in range(19, 27):  # 8 líneas de puertas
        line = lines[i].strip().split()
        r1, c1, r2, c2 = map(int, line)
        doors.append(((r1 - 1, c1 - 1), (r2 - 1, c2 - 1)))  # Ajustar índices a 0-based

    # Leer los puntos de entrada
    entry_points = []
    for i in range(27, 31):  # 4 líneas de puntos de entrada
        line = lines[i].strip().split()
        row, col = int(line[0]), int(line[1])
        entry_points.append((row - 1, col - 1))  # Ajustar índices a 0-based

    return walls, markers, fires, doors, entry_points


In [11]:
walls, markers, fires, doors, entry_points = load_scenario("final.txt")
model = FireRescueModel(walls, markers, fires, doors, entry_points)
Max_steps = 10000

while not model.check_game_end() and Max_steps > 0:
  model.step()
  Max_steps -=1

print("Steps: ", model.steps)


Error collecting data: 'NoneType' object is not subscriptable
Error in reporter Grid: 'NoneType' object is not subscriptable
Steps: 1
Damage: 0
VictimsRescued: 0
VictimsLost: 0
Error collecting data: 'NoneType' object is not subscriptable
Error in reporter Grid: 'NoneType' object is not subscriptable
Steps: 2
Damage: 0
VictimsRescued: 0
VictimsLost: 0
Error collecting data: 'NoneType' object is not subscriptable
Error in reporter Grid: 'NoneType' object is not subscriptable
Steps: 3
Damage: 0
VictimsRescued: 0
VictimsLost: 0
Error collecting data: 'NoneType' object is not subscriptable
Error in reporter Grid: 'NoneType' object is not subscriptable
Steps: 4
Damage: 0
VictimsRescued: 0
VictimsLost: 0
Error collecting data: 'NoneType' object is not subscriptable
Error in reporter Grid: 'NoneType' object is not subscriptable
Steps: 5
Damage: 0
VictimsRescued: 0
VictimsLost: 0
Error collecting data: 'NoneType' object is not subscriptable
Error in reporter Grid: 'NoneType' object is not subs